# Introduction of PETSc Analyzer

The objective of the PETSc analyzer is to validate coding style conventions included in the
[PETSc Style and Usage Guide](https://petsc.org/release/developers/style/).

## Rules about C Usage

### Rule-1, Rule-2 and Rule-3

1. Array and pointer arguments where the array values are not changed should be labeled as const arguments.
2. Scalar values passed to functions should never be labeled as const.
3. Subroutines that would normally have a void** argument to return a pointer to some data should actually be prototyped as void*. This prevents the caller from having to put a (void**) cast in each function call.

### Sample C Code (PETSc_Rules_1-3.c)

void myfunc_first (int * param1,  // object is modified
             const int * param2,
                   int * param3, // Noncompliant for Rule-1
                   int * param4) // Noncompliant for Rule-1
{
  *param1 = *param2 + *param3 + *param4;
}

void myfunc_second (      int  param1,  // object is modified
             const int param2,  // Noncompliant for Rule-2
             const int param3, // Noncompliant for Rule-2
                   int param4)
{
  param1 = param2 + param3 + param4;
}

void myfunc_third(void** ptr)  //Noncomplaint for Rule-3
{
        printf("str: %s\n",*ptr);
}

void myfunc_fourth(void **ptr)  //Noncomplaint for Rule-3
{
        printf("str: %s\n",*ptr);
}

int main (int argc,
          const char **argv)
{
  return argc;
}

### Script for Analyzing Rule-1, Rule-2 and Rule-3

In [3]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
Config.set_library_path('/usr/local/Cellar/llvm/11.0.0/lib')
Function_List=[]
Function_Param_List=[]

def extract_function_list(tu):
    filename = tu.cursor.spelling
    func_name=""
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
           if c.spelling not in Function_List:
                Function_List.append(c.spelling)
                func_name=c.spelling
       elif c.kind == clang.cindex.CursorKind.PARM_DECL:
            param_lin, param_col=extract_line_column(c)
            param_col=param_col[:-1]
            col=[]
            col.append(func_name)
            col.append(c.spelling)
            col.append(0) # Wheather parameter is a scaler or pointer variable
            col.append(0) # Flag to check the paramter is const or not
            col.append(0) # Check paramter has been used on LHS od an experision
            col.append(0) # to check wheather paramter type is void with * or **"
            col.append(param_lin) # line number
            col.append(param_col) #Column Number
            Function_Param_List.append(col)

def extract_line_column(c):
    x=str(c.location)
    y=x.split(',')
    line= y[1]
    column=y[2]
    line=line.split()
    line=line[1]
    column=column.split()
    column=column[1]
    return line, column
    
def Scan_Func_Param_List(func_name, param):
    index=-1
    cnt=0
    for f in Function_Param_List:
        fun_name=f[0]
        parm=f[1]
        if fun_name==func_name and parm==param:
            index=cnt
        cnt=cnt+1
    return index
def Traverse(tu):
    filename = tu.cursor.spelling
    const_bool=0
    double_ast_bool=0
    ast_bool=0
    void_bool=0
    void_bool_cont=0
    void_st=""
    param_name=""
    func_name=""
    param_lin=0
    param_col=0
    void_lin=0
    void_col=0
    ast_lin=0
    ast_col=0
    double_ast_lin=0
    double_ast_col=0
    double_ast=""
    double_ast_lin1=0
    double_ast_lin=0
    double_ast_col1=0
    double_ast_col2=0
    for c in tu.cursor.get_tokens():
        filename = tu.cursor.spelling
        if c.location.file is None:
            pass
        elif c.location.file.name != filename:
            pass
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and c.spelling in Function_List:
            if c.cursor.kind != clang.cindex.CursorKind.DECL_REF_EXPR and  c.cursor.kind != clang.cindex.CursorKind.CALL_EXPR and  c.cursor.kind != clang.cindex.CursorKind.OVERLOADED_DECL_REF:
                func_name=c.spelling
        elif c.kind==clang.cindex.TokenKind.KEYWORD and c.spelling =="const":
                const_bool=1
        elif c.kind==clang.cindex.TokenKind.KEYWORD and c.spelling == "void":
                void_bool_cont=1
                void_st="void"
                void_lin, void_col=extract_line_column(c)
                void_col=void_col[:-1]
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and c.spelling not in Function_List:
            index=Scan_Func_Param_List(func_name, c.spelling)
            if index!=-1:
                param_name=c.spelling
                param_lin, param_col=extract_line_column(c)
                param_col=param_col[:-1]
            if index!=-1 and const_bool==1:
                Function_Param_List[index][3]=1
                const_bool=0
            if index!=-1 and ast_bool==1:
                Function_Param_List[index][2]=1
                ast_bool=0
            if index!=-1 and void_bool==1:
                Function_Param_List[index][5]=1
                void_bool=0
            if index!=-1 and double_ast_bool==1 and Function_Param_List[index][5]!=1 :
                Function_Param_List[index][5]=1
                double_ast_bool=0
                double_ast=""
        elif   c.kind==clang.cindex.TokenKind.PUNCTUATION:
            if c.spelling =="*":
                double_ast=double_ast+"*"
                if double_ast=="*":
                    double_ast_lin1, double_ast_col1 = extract_line_column(c)
                    double_ast_col1=double_ast_col1[:-1]
                elif double_ast=="**":
                    double_ast_lin2, double_ast_col2 = extract_line_column(c)
                    double_ast_col2=double_ast_col2[:-1]
                    double_ast=""
                    if  int(double_ast_col1)+1 == int(double_ast_col2):
                        double_ast_bool=1
                    #double_ast=""
                ast_lin, ast_col= extract_line_column(c)
                ast_col=ast_col[:-1]
                ast_bool=1
                if (int(ast_col)==int(void_col)+4) and void_bool_cont==1:
                    void_st=void_st+"*"
                    if len(void_st)==5:
                        void_bool=1
                        void_bool_cont=0
                        void_st=""
            if c.spelling=="=":
                  equal_lin, equal_col= extract_line_column(c)
                  equal_col=equal_col[:-1]
                  if int(param_lin)-1<=int(equal_lin) and equal_col<param_col and param_name!="":
                      index=Scan_Func_Param_List(func_name, param_name)
                      if index!=-1:
                           Function_Param_List[index][4]=1

idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("PETSc_Rules_1-3.c", args='-xc++ --std=c++11'.split())
extract_function_list(tu)
Traverse(tu)

print ("Function_Name,  Paramter_Name,  Line_Number,   Column_Number")
print ("------------------------------------------------------------")
print ("Non-compliant Parameters for Rule-1")
print ("-----------------------------------")
for f in Function_Param_List:
    if (f[2]==1 and f[3]==0 and f[4]==0):
        print (f[0], f[1], f[6], f[7])
print ("-----------------------------------")
print ("Non-compliant Parameters for Rule-2")
print ("------------------------------------")
for f in Function_Param_List:
    if f[2]==0 and f[3]==1:
        print (f[0], f[1], f[6], f[7])
print ("-----------------------------------")
print ("Non-compliant paramters for Rule-3")
print ("---------------------------------")
for f in Function_Param_List:
    if f[5]==1:
        print (f[0], f[1], f[6], f[7])
print ("-----------------------------------")

Function_Name,  Paramter_Name,  Line_Number,   Column_Number
------------------------------------------------------------
Non-compliant Parameters for Rule-1
-----------------------------------
myfunc_first param3 3 26
myfunc_first param4 4 26
myfunc_third ptr 16 26
myfunc_fourth ptr 20 27
-----------------------------------
Non-compliant Parameters for Rule-2
------------------------------------
myfunc_second param2 10 24
myfunc_second param3 11 24
-----------------------------------
Non-compliant paramters for Rule-3
---------------------------------
myfunc_third ptr 16 26
myfunc_fourth ptr 20 27
-----------------------------------


### Rule-4

       Do not use the register directive.

### Sample C Code (PETSc_Rules_4.c)


#include <stdio.h>
  
/* function declaration */
void func(void);

static int count = 5; /* global variable */
register int x;

main() {

   while(count--) {
      func();
   }

   return 0;
}

/* function definition */
void func( void ) {

   static int i = 5; /* local static variable */
   i++;

   printf("i is %d and count is %d\n", i, count);
}


### Script for Analyzing Rule-4

In [12]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
#Config.set_library_path('/usr/local/Cellar/llvm/11.0.0/lib')
Function_List=[]
Function_Register_Variables=[]
File_Content_Array=[]

def file_to_array(cursor):
    filename = cursor.spelling
    with open(filename) as f:
        for line in f:
            File_Content_Array.append(line)

def extract_function_list(tu):
    filename = tu.cursor.spelling
    func_name=""
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            if c.spelling not in Function_List:
                Function_List.append(c.spelling)

def extract_line_column(c):
    x=str(c.location)
    y=x.split(',')
    line= y[1]
    column=y[2]
    line=line.split()
    line=line[1]
    column=column.split()
    column=column[1]
    return line, column
def Traverse(tu):
    filename = tu.cursor.spelling
    register_bool=0
    register_lin=0
    register_col=0
    variable_lin=0
    variable_col=0
    param_name=""
    for c in tu.cursor.get_tokens():
        filename = tu.cursor.spelling
        if c.location.file is None:
            pass
        elif c.location.file.name != filename:
            pass
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and c.spelling in Function_List:
            if c.cursor.kind != clang.cindex.CursorKind.DECL_REF_EXPR and  c.cursor.kind != clang.cindex.CursorKind.CALL_EXPR and  c.cursor.kind != clang.cindex.CursorKind.OVERLOADED_DECL_REF:
                func_name=c.spelling
        elif c.kind==clang.cindex.TokenKind.KEYWORD and c.spelling =="register":
            register_bool=1
            register_lin, register_col = extract_line_column(c)
            regsiter_col = register_col[:-1]
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and c.spelling not in Function_List:
             variable_lin, variable_col = extract_line_column(c)
             variable_col= variable_col[:-1]
             if int(register_lin)+1 <= int(variable_col) and register_bool==1:
                 col=[]
                 col.append(func_name)
                 col.append(c.spelling)
                 st=File_Content_Array[int(variable_lin)-1]
                 col.append(st.strip())
                 col.append(variable_lin)
                 col.append(variable_col)
                 Function_Register_Variables.append(col)
                 register_bool=0
idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("PETSc_Rule_4.c", args='-xc++ --std=c++11'.split())
file_to_array(tu.cursor)
extract_function_list(tu)
Traverse(tu)
print ("Function_Name, Variable_Name, Definition, Line_Number, Column_Number")
print ("--------------------------------------------------------------------")
for f in Function_Register_Variables:
    print (f)




Function_Name, Variable_Name, Definition, Line_Number, Column_Number
--------------------------------------------------------------------
['func', 'x', 'register int x;', '8', '14']
['func', 'i', 'printf("i is %d and count is %d\\n", i, count);', '26', '40']


### Rule-5

       Do not use if (rank == 0) or if (v == NULL) or if (flg == PETSC_TRUE) or if (flg == PETSC_FALSE). Instead, use if (!rank) or if (!v) or if (flg) or if (!flg).

### Sample C Code (PETSc_Rules_5.c)

#include <stdio.h>
/* function declaration */
int func(int);
main() {
   int a=4, b=6, flag;
   flag=func(a);
   if (flag==0)
          printf("a is EVEN")
   else
          print ("a is ODD");
   flag=func(b)
   if (flag)
          printf("b is EVEN");
   else
          printf("b is ODD");
   return 0;
}
/* function definition */
int func(int a) {
    if(a%2==0)
        return 0;
    else
        return 1;
}

### Script for Analyzing Rule-5

In [17]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
#Config.set_library_path('/usr/local/Cellar/llvm/11.0.0/lib')
Function_List=[]
Function_Condit_Variables=[]
File_Content_Array=[]

def file_to_array(cursor):
    filename = cursor.spelling
    with open(filename) as f:
        for line in f:
            File_Content_Array.append(line)
  
def extract_function_list(tu):
    filename = tu.cursor.spelling
    func_name=""
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            if c.spelling not in Function_List:
                 Function_List.append(c.spelling)

def extract_line_column(c):
    x=str(c.location)
    y=x.split(',')
    line= y[1]
    column=y[2]
    line=line.split()
    line=line[1]
    column=column.split()
    column=column[1]
    return line, column
def Traverse(tu):
    filename = tu.cursor.spelling
    condit_bool=0
    condit_lin=0
    condit_col=0
    func_name=""
    for c in tu.cursor.get_tokens():
        filename = tu.cursor.spelling
        if c.location.file is None:
            pass
        elif c.location.file.name != filename:
            pass
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and  c.spelling in Function_List:
            if c.cursor.kind != clang.cindex.CursorKind.DECL_REF_EXPR and  c.cursor.kind != clang.cindex.CursorKind.CALL_EXPR and  c.cursor.kind != clang.cindex.CursorKind.OVERLOADED_DECL_REF:
                func_name=c.spelling

        elif c.kind==clang.cindex.TokenKind.PUNCTUATION and c.spelling =="==":
            condit_lin, condit_col = extract_line_column(c)
            condit_col = condit_col[:-1]
            col=[]
            col.append(func_name)
            st=File_Content_Array[int(condit_lin)-1]
            col.append(st.strip())
            col.append(condit_lin)
            col.append(condit_col)
            Function_Condit_Variables.append(col)




           # print c.spelling, c.kind, c.location

idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("PETSc_Rule_5.c", args='-xc++ --std=c++11'.split())
file_to_array(tu.cursor)
extract_function_list(tu)
Traverse(tu)
print ("Function_Name, Definition, Line_Number, Column_Number")
print ("-----------------------------------------------------")
for f in Function_Condit_Variables:
    print (f)


Function_Name, Definition, Line_Number, Column_Number
-----------------------------------------------------
['main', 'if (flag==0)', '7', '12']
['func', 'if(a%2==0)', '20', '11']


### Rule-6

      Do not use #ifdef or #ifndef. Rather, use #if defined(... or #if !defined

### Sample C Code (PETSc_Rules_6.c)

#include <stdio.h>
#define WINDOWS 1
int main()
{
   #ifdef WINDOWS
       printf("Windows");
   #endif
   #if WINDOWS
      printf("Windows ");
   #endif
   #ifndef MESSAGE
      #define MESSAGE "You wish!"
   #endif
   #if !defined  MESSAGE
      #define MESSAGE "You wish!"
   #endif
   return 0;
}


### Script for Analyzing Rule-6

In [23]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
#Config.set_library_path('/usr/local/Cellar/llvm/11.0.0/lib')
Function_List=[]
Function_Define_Valid=[]
Function_Define_Invalid=[]
def extract_line_column(c):
    x=str(c.location)
    y=x.split(',')
    line= y[1]
    column=y[2]
    line=line.split()
    line=line[1]
    column=column.split()
    column=column[1]
    return line, column

def Traverse(tu):
    filename = tu.cursor.spelling
    define_bool=0
    define_lin=0
    define_col=0
    define_name=""
    for c in tu.cursor.get_tokens():
        filename = tu.cursor.spelling
        if c.location.file is None:
            pass
        elif c.location.file.name != filename:
            pass
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and  c.spelling in Function_List:
            if c.cursor.kind != clang.cindex.CursorKind.DECL_REF_EXPR and  c.cursor.kind != clang.cindex.CursorKind.CALL_EXPR and  c.cursor.kind != clang.cindex.CursorKind.OVERLOADED_DECL_REF:
                func_name=c.spelling
        elif c.kind==clang.cindex.TokenKind.PUNCTUATION and c.spelling=="#":
            define_bool=1
            define_name= c.spelling
        elif c.kind==clang.cindex.TokenKind.KEYWORD and c.spelling=="if" and define_bool==1:
            define_name=define_name+c.spelling
            define_bool=0
            define_lin, define_col = extract_line_column(c)
            define_col = define_col[:-1]
            col=[]
            col.append(func_name)
            col.append(define_name)
            col.append(define_lin)
            col.append(define_col)
            Function_Define_Valid.append(col)
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and (c.spelling =="ifdef" or c.spelling =="ifndef"):
            define_lin, define_col = extract_line_column(c)
            define_col = define_col[:-1]
            col=[]
            col.append(func_name)
            col.append(c.spelling)
            col.append(define_lin)
            col.append(define_col)
            Function_Define_Invalid.append(col)
            
idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("PETSc_Rule_6.c", args='-xc++ --std=c++11'.split())
extract_function_list(tu)
Traverse(tu)
print ("Function_Name, Definition, Line_Number, Column_Number")
print ("-----------------------------------------------------")
print ("Compliant preprocessors")
for f in Function_Define_Valid:
    print (f)
print ("Non-Compliant Preprocessor")
for f in Function_Define_Invalid:
    print (f)



Function_Name, Definition, Line_Number, Column_Number
-----------------------------------------------------
Compliant preprocessors
['main', '#if', '8', '5']
['main', '#if', '14', '5']
Non-Compliant Preprocessor
['main', 'ifdef', '5', '5']
['main', 'ifndef', '11', '5']


### Rule-7

      Never use system random number generators such as rand() in PETSc code or examples because these can produce different results on different systems thus making portability testing difficult. 

### Sample C Code (PETSc_Rules_7.c)


#include <stdio.h>
#include <stdlib.h>
int main () {
   int i, n;
   time_t t;
   n = 5;
   /* Intializes random number generator */
   srand((unsigned) time(&t));
   /* Print 5 random numbers from 0 to 49 */
   for( i = 0 ; i < n ; i++ ) {
      printf("%d\n", rand() % 50);
   }
   printf("%d", rand());
   return(0);
}

### Script for Analyzing Rule-7

In [31]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
#Config.set_library_path('/usr/local/Cellar/llvm/11.0.0/lib')
Function_List=[]
Function_Rand_Usage=[]
File_Content_Array=[]

def file_to_array(cursor):
    filename = cursor.spelling
    with open(filename) as f:
        for line in f:
            File_Content_Array.append(line)

def extract_function_list(tu):
    filename = tu.cursor.spelling
    func_name=""
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            if c.spelling not in Function_List:
                 Function_List.append(c.spelling)

def extract_line_column(c):
    x=str(c.location)
    y=x.split(',')
    line= y[1]
    column=y[2]
    line=line.split()
    line=line[1]
    column=column.split()
    column=column[1]
    return line, column
def Traverse(tu):
    filename = tu.cursor.spelling
    rand_bool=0
    rand_lin=0
    rand_col=0
    paren_lin=0
    paren_col=0
    rand_name=""
    for c in tu.cursor.get_tokens():
        filename = tu.cursor.spelling
        if c.location.file is None:
            pass
        elif c.location.file.name != filename:
            pass
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and  c.spelling in Function_List:
            if c.cursor.kind != clang.cindex.CursorKind.DECL_REF_EXPR and  c.cursor.kind != clang.cindex.CursorKind.CALL_EXPR and  c.cursor.kind != clang.cindex.CursorKind.OVERLOADED_DECL_REF:
                func_name=c.spelling
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and c.spelling=="rand":
            rand_bool=1
            rand_name= c.spelling
            rand_lin, rand_col = extract_line_column(c)
            rand_col = rand_col[:-1]
        elif c.kind==clang.cindex.TokenKind.PUNCTUATION and c.spelling=="(" and rand_bool==1:
            rand_bool=0
            paren_lin, paren_col = extract_line_column(c)
            paren_col = paren_col[:-1]
            if int(rand_col)+4==int(paren_col):
                col=[]
                col.append(func_name)
                st=File_Content_Array[int(rand_lin)-1]
                col.append(st.strip())
                col.append(rand_lin)
                col.append(rand_col)
                Function_Rand_Usage.append(col)
idx=clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("PETSc_Rule_7.c", args='-xc++ --std=c++11'.split())
file_to_array(tu.cursor)
extract_function_list(tu)
Traverse(tu)
print ("Function_Name, Definition, Line_Number, Column_Number")
print ("-----------------------------------------------------")
for f in Function_Rand_Usage:
    print (f)



Function_Name, Definition, Line_Number, Column_Number
-----------------------------------------------------
['main', 'printf("%d\\n", rand() % 50);', '14', '22']
['main', 'printf("%d", rand());', '16', '17']
